# fhv_tripdata_2015/2016

In [10]:
# fetch fhv_tripdata_2015/2016

import csv
import sqlite3
import pandas as pd

conn = sqlite3.connect('uber_sample.db')
cur = conn.cursor()

for y in range (5, 7):
    
    cur.execute('DROP TABLE IF EXISTS fhv_tripdata_201' + str(y))
    cur.execute('CREATE TABLE fhv_tripdata_201' + str(y) + '(Dispatching_base_num TEXT, Pickup_date TEXT, locationID INTEGER)')
    
    for m in range (1, 13):

        if m < 10:
            m = "0" + str(m)
        else:
            m = str(m)

        url = 'fhv_tripdata_201' + str(y) + '-' + m + '_s.csv'
        #url = 'https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-01.csv'
        print(url)

        try:
            df = pd.read_csv(url)

        except:
            continue

        else:

            data = df.transpose().to_dict().values()

            for i in data:
                cur.execute('INSERT INTO fhv_tripdata_2015 (Dispatching_base_num, Pickup_date, locationID) VALUES (?, ?, ?)', 
                            (i['Dispatching_base_num'], i['Pickup_date'], i['locationID']))
          
conn.commit()
cur.close()

fhv_tripdata_2015-01_s.csv
fhv_tripdata_2015-02_s.csv
fhv_tripdata_2015-03_s.csv
fhv_tripdata_2015-04_s.csv
fhv_tripdata_2015-05_s.csv
fhv_tripdata_2015-06_s.csv
fhv_tripdata_2015-07_s.csv
fhv_tripdata_2015-08_s.csv
fhv_tripdata_2015-09_s.csv
fhv_tripdata_2015-10_s.csv
fhv_tripdata_2015-11_s.csv
fhv_tripdata_2015-12_s.csv
fhv_tripdata_2016-01_s.csv
fhv_tripdata_2016-02_s.csv
fhv_tripdata_2016-03_s.csv
fhv_tripdata_2016-04_s.csv
fhv_tripdata_2016-05_s.csv
fhv_tripdata_2016-06_s.csv
fhv_tripdata_2016-07_s.csv
fhv_tripdata_2016-08_s.csv
fhv_tripdata_2016-09_s.csv
fhv_tripdata_2016-10_s.csv
fhv_tripdata_2016-11_s.csv
fhv_tripdata_2016-12_s.csv


# hourly count 2015/16

In [3]:
# fhv_tripdata_hourly_count

import csv
import sqlite3
import pandas as pd
import math

conn = sqlite3.connect('uber_sample.db')
cur = conn.cursor()

for y in range (5, 6):
    
    cur.execute('DROP TABLE IF EXISTS fhv_tripdata_hourly')
    cur.execute('CREATE TABLE fhv_tripdata_hourly (Dispatching_base_num TEXT, Pickup_time TEXT, locationID INTEGER ,count INTEGER)')
    
    for m in range (1, 2):

        if m < 10:
            m = "0" + str(m)
        else:
            m = str(m)

        url = 'fhv_tripdata_201' + str(y) + '-' + m + '_s.csv'
        #url = 'https://s3.amazonaws.com/nyc-tlc/trip+data/fhv_tripdata_2015-01.csv'
        print(url)

        try:
            df = pd.read_csv(url)

        except:
            continue

        else:

            data = df.transpose().to_dict().values()

            for i in data:
                
                base = i['Dispatching_base_num']
                time_pieces = i['Pickup_date'].split(":")
                time = time_pieces[0]
                loc = i['locationID']
                
                if math.isnan(loc):
                    loc = 999
                
                cur.execute('''SELECT count FROM fhv_tripdata_hourly 
                            WHERE Dispatching_base_num = ?  AND Pickup_time = ? and locationID = ?''', 
                            (base, time, loc))
                row = cur.fetchone()
                
                if row is None:
                    cur.execute('''INSERT INTO fhv_tripdata_hourly (Dispatching_base_num, Pickup_time, locationID, count)
                                VALUES (?, ?, ?, 1)''', (base, time, loc))
                    
                else:
                    cur.execute('''UPDATE fhv_tripdata_hourly SET count = count + 1 
                                WHERE Dispatching_base_num = ?  AND Pickup_time = ? AND locationID = ?''',
                                (base, time, loc))

          
conn.commit()
cur.close()

fhv_tripdata_2015-01_s.csv


# Assign Location ID

In [35]:
import shapegeocode
gc = shapegeocode.geocoder('taxi_zones_wgs84.shp')
type(gc.geocode(-73.9, 40.7))


NoneType

In [26]:
import shapegeocode
gc = shapegeocode.geocoder('NUTS_RG_03M_2006.shp')
gc.geocode(100, 20, max_dist = 10000)


{'AREA': 0.0,
 'LEN': 0.0,
 'NUTS_ID': 'FR',
 'OBJECTID': 30,
 'STAT_LEVL_': 0,
 'Shape_Area': 71.7607845062,
 'Shape_Leng': 117.268578339}

In [ ]:
import networkx as nx
import numpy as np
import igraph as ig
import shapefile
import csv
import qgis.core
qgis.utils.iface
import matplotlib.pyplot as plt
import processing

from PyQt4.QtCore import QVariant
from qgis.core import QgsField, QgsExpression, QgsFeature

#Clear Canvas
QgsMapLayerRegistry.instance().removeAllMapLayers()

#Set CRS
canvas = iface.mapCanvas()
selectedcrs="EPSG:2163"
target_crs = QgsCoordinateReferenceSystem()
target_crs.createFromUserInput(selectedcrs)
canvas.setDestinationCrs(target_crs)

#Set Paths
boundarypath = "/Volumes/GoogleDrive/My Drive/RRAbandonment/Additional Maps/County_Boundaries/"
statapath ="/Volumes/GoogleDrive/My Drive/RRAbandonment/Additional Maps/Highways/"
hwypath = "/Users/dustball/Dropbox/gis files/interstates/annualHighwayFinalFiles/annualHwyFinalCountyDissolved/"

#Add County Base Shapefile - Year 2000
mastercountylayer = iface.addVectorLayer(boundarypath+"Master20001231Boundaries.shp", "Master2000Boundaries", "ogr")
if not mastercountylayer:
  print "Layer failed to load!"

#1954
#Add Dissolved Highway Files
IHS1954final_dissolved = iface.addVectorLayer(hwypath + "IHS1954final_dissolved.shp", "IHS1954final_dissolved", "ogr")
if not IHS1954final_dissolved:
  print "Layer failed to load!"  
  
#Intersect Hwy Layer and County Boundary Layer
Interstate1954_2000Boundary = processing.runalg('qgis:intersection', IHS1954final_dissolved, mastercountylayer, hwypath+"Interstate1954_2000Boundary.shp")

Interstate1954_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1954_2000Boundary.shp","Interstate1954_2000Boundary", "ogr")
if not Interstate1954_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1954_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1954_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1954_2000Boundary.updateFields()
idx = Interstate1954_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1954_2000Boundary.pendingFields() )

for f in Interstate1954_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1954_2000Boundary.updateFeature( f )

Interstate1954_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1954_2000Boundary,statapath+"Interstate1954_2000Boundary.csv", "utf-8", None, "CSV")

#1956
#Add Dissolved Highway Files
IHS1956final_dissolved = iface.addVectorLayer(hwypath + "IHS1956final_dissolved.shp", "IHS1956final_dissolved", "ogr")
if not IHS1956final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1956_2000Boundary = processing.runalg('qgis:intersection', IHS1956final_dissolved, mastercountylayer, hwypath+"Interstate1956_2000Boundary.shp")

Interstate1956_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1956_2000Boundary.shp","Interstate1956_2000Boundary", "ogr")
if not Interstate1956_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1956_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1956_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1956_2000Boundary.updateFields()
idx = Interstate1956_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1956_2000Boundary.pendingFields() )

for f in Interstate1956_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1956_2000Boundary.updateFeature( f )

Interstate1956_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1956_2000Boundary,statapath+"Interstate1956_2000Boundary.csv", "utf-8", None, "CSV")

#1958
#Add Dissolved Highway Files
IHS1958final_dissolved = iface.addVectorLayer(hwypath + "IHS1958final_dissolved.shp", "IHS1958final_dissolved", "ogr")
if not IHS1958final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1958_2000Boundary = processing.runalg('qgis:intersection', IHS1958final_dissolved, mastercountylayer, hwypath+"Interstate1958_2000Boundary.shp")

Interstate1958_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1958_2000Boundary.shp","Interstate1958_2000Boundary", "ogr")
if not Interstate1958_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1958_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1958_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1958_2000Boundary.updateFields()
idx = Interstate1958_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1958_2000Boundary.pendingFields() )

for f in Interstate1958_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1958_2000Boundary.updateFeature( f )

Interstate1958_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1958_2000Boundary,statapath+"Interstate1958_2000Boundary.csv", "utf-8", None, "CSV")

#1959
#Add Dissolved Highway Files
IHS1959final_dissolved = iface.addVectorLayer(hwypath + "IHS1959final_dissolved.shp", "IHS1959final_dissolved", "ogr")
if not IHS1959final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1959_2000Boundary = processing.runalg('qgis:intersection', IHS1959final_dissolved, mastercountylayer, hwypath+"Interstate1959_2000Boundary.shp")

Interstate1959_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1959_2000Boundary.shp","Interstate1959_2000Boundary", "ogr")
if not Interstate1959_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1959_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1959_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1959_2000Boundary.updateFields()
idx = Interstate1959_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1959_2000Boundary.pendingFields() )

for f in Interstate1959_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1959_2000Boundary.updateFeature( f )

Interstate1959_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1959_2000Boundary,statapath+"Interstate1959_2000Boundary.csv", "utf-8", None, "CSV")

#1963
#Add Dissolved Highway Files
IHS1963final_dissolved = iface.addVectorLayer(hwypath + "IHS1963final_dissolved.shp", "IHS1963final_dissolved", "ogr")
if not IHS1963final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1963_2000Boundary = processing.runalg('qgis:intersection', IHS1963final_dissolved, mastercountylayer, hwypath+"Interstate1963_2000Boundary.shp")

Interstate1963_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1963_2000Boundary.shp","Interstate1963_2000Boundary", "ogr")
if not Interstate1963_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1963_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1963_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1963_2000Boundary.updateFields()
idx = Interstate1963_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1963_2000Boundary.pendingFields() )

for f in Interstate1963_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1963_2000Boundary.updateFeature( f )

Interstate1963_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1963_2000Boundary,statapath+"Interstate1963_2000Boundary.csv", "utf-8", None, "CSV")

#1964
#Add Dissolved Highway Files
IHS1964final_dissolved = iface.addVectorLayer(hwypath + "IHS1964final_dissolved.shp", "IHS1964final_dissolved", "ogr")
if not IHS1964final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1964_2000Boundary = processing.runalg('qgis:intersection', IHS1964final_dissolved, mastercountylayer, hwypath+"Interstate1964_2000Boundary.shp")

Interstate1964_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1964_2000Boundary.shp","Interstate1964_2000Boundary", "ogr")
if not Interstate1964_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1964_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1964_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1964_2000Boundary.updateFields()
idx = Interstate1964_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1964_2000Boundary.pendingFields() )

for f in Interstate1964_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1964_2000Boundary.updateFeature( f )

Interstate1964_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1964_2000Boundary,statapath+"Interstate1964_2000Boundary.csv", "utf-8", None, "CSV")

#1967
#Add Dissolved Highway Files
IHS1967final_dissolved = iface.addVectorLayer(hwypath + "IHS1967final_dissolved.shp", "IHS1967final_dissolved", "ogr")
if not IHS1967final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1967_2000Boundary = processing.runalg('qgis:intersection', IHS1967final_dissolved, mastercountylayer, hwypath+"Interstate1967_2000Boundary.shp")

Interstate1967_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1967_2000Boundary.shp","Interstate1967_2000Boundary", "ogr")
if not Interstate1967_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1967_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1967_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1967_2000Boundary.updateFields()
idx = Interstate1967_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1967_2000Boundary.pendingFields() )

for f in Interstate1967_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1967_2000Boundary.updateFeature( f )

Interstate1967_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1967_2000Boundary,statapath+"Interstate1967_2000Boundary.csv", "utf-8", None, "CSV")

#1969
#Add Dissolved Highway Files
IHS1969final_dissolved = iface.addVectorLayer(hwypath + "IHS1969final_dissolved.shp", "IHS1969final_dissolved", "ogr")
if not IHS1969final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1969_2000Boundary = processing.runalg('qgis:intersection', IHS1969final_dissolved, mastercountylayer, hwypath+"Interstate1969_2000Boundary.shp")

Interstate1969_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1969_2000Boundary.shp","Interstate1969_2000Boundary", "ogr")
if not Interstate1969_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1969_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1969_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1969_2000Boundary.updateFields()
idx = Interstate1969_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1969_2000Boundary.pendingFields() )

for f in Interstate1969_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1969_2000Boundary.updateFeature( f )

Interstate1969_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1969_2000Boundary,statapath+"Interstate1969_2000Boundary.csv", "utf-8", None, "CSV")

#1972
#Add Dissolved Highway Files
IHS1972final_dissolved = iface.addVectorLayer(hwypath + "IHS1972final_dissolved.shp", "IHS1972final_dissolved", "ogr")
if not IHS1972final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1972_2000Boundary = processing.runalg('qgis:intersection', IHS1972final_dissolved, mastercountylayer, hwypath+"Interstate1972_2000Boundary.shp")

Interstate1972_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1972_2000Boundary.shp","Interstate1972_2000Boundary", "ogr")
if not Interstate1972_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1972_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1972_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1972_2000Boundary.updateFields()
idx = Interstate1972_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1972_2000Boundary.pendingFields() )

for f in Interstate1972_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1972_2000Boundary.updateFeature( f )

Interstate1972_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1972_2000Boundary,statapath+"Interstate1972_2000Boundary.csv", "utf-8", None, "CSV")

#1974
#Add Dissolved Highway Files
IHS1974final_dissolved = iface.addVectorLayer(hwypath + "IHS1974final_dissolved.shp", "IHS1974final_dissolved", "ogr")
if not IHS1974final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1974_2000Boundary = processing.runalg('qgis:intersection', IHS1974final_dissolved, mastercountylayer, hwypath+"Interstate1974_2000Boundary.shp")

Interstate1974_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1974_2000Boundary.shp","Interstate1974_2000Boundary", "ogr")
if not Interstate1974_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1974_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1974_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1974_2000Boundary.updateFields()
idx = Interstate1974_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1974_2000Boundary.pendingFields() )

for f in Interstate1974_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1974_2000Boundary.updateFeature( f )

Interstate1974_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1974_2000Boundary,statapath+"Interstate1974_2000Boundary.csv", "utf-8", None, "CSV")

#1976
#Add Dissolved Highway Files
IHS1976final_dissolved = iface.addVectorLayer(hwypath + "IHS1976final_dissolved.shp", "IHS1976final_dissolved", "ogr")
if not IHS1976final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1976_2000Boundary = processing.runalg('qgis:intersection', IHS1976final_dissolved, mastercountylayer, hwypath+"Interstate1976_2000Boundary.shp")

Interstate1976_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1976_2000Boundary.shp","Interstate1976_2000Boundary", "ogr")
if not Interstate1976_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1976_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1976_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1976_2000Boundary.updateFields()
idx = Interstate1976_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1976_2000Boundary.pendingFields() )

for f in Interstate1976_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1976_2000Boundary.updateFeature( f )

Interstate1976_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1976_2000Boundary,statapath+"Interstate1976_2000Boundary.csv", "utf-8", None, "CSV")

#1977
#Add Dissolved Highway Files
IHS1977final_dissolved = iface.addVectorLayer(hwypath + "IHS1977final_dissolved.shp", "IHS1977final_dissolved", "ogr")
if not IHS1977final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1977_2000Boundary = processing.runalg('qgis:intersection', IHS1977final_dissolved, mastercountylayer, hwypath+"Interstate1977_2000Boundary.shp")

Interstate1977_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1977_2000Boundary.shp","Interstate1977_2000Boundary", "ogr")
if not Interstate1977_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1977_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1977_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1977_2000Boundary.updateFields()
idx = Interstate1977_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1977_2000Boundary.pendingFields() )

for f in Interstate1977_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1977_2000Boundary.updateFeature( f )

Interstate1977_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1977_2000Boundary,statapath+"Interstate1977_2000Boundary.csv", "utf-8", None, "CSV")

#1978
#Add Dissolved Highway Files
IHS1978final_dissolved = iface.addVectorLayer(hwypath + "IHS1978final_dissolved.shp", "IHS1978final_dissolved", "ogr")
if not IHS1978final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1978_2000Boundary = processing.runalg('qgis:intersection', IHS1978final_dissolved, mastercountylayer, hwypath+"Interstate1978_2000Boundary.shp")

Interstate1978_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1978_2000Boundary.shp","Interstate1978_2000Boundary", "ogr")
if not Interstate1978_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1978_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1978_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1978_2000Boundary.updateFields()
idx = Interstate1978_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1978_2000Boundary.pendingFields() )

for f in Interstate1978_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1978_2000Boundary.updateFeature( f )

Interstate1978_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1978_2000Boundary,statapath+"Interstate1978_2000Boundary.csv", "utf-8", None, "CSV")

#1979
#Add Dissolved Highway Files
IHS1979final_dissolved = iface.addVectorLayer(hwypath + "IHS1979final_dissolved.shp", "IHS1979final_dissolved", "ogr")
if not IHS1979final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1979_2000Boundary = processing.runalg('qgis:intersection', IHS1979final_dissolved, mastercountylayer, hwypath+"Interstate1979_2000Boundary.shp")

Interstate1979_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1979_2000Boundary.shp","Interstate1979_2000Boundary", "ogr")
if not Interstate1979_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1979_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1979_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1979_2000Boundary.updateFields()
idx = Interstate1979_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1979_2000Boundary.pendingFields() )

for f in Interstate1979_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1979_2000Boundary.updateFeature( f )

Interstate1979_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1979_2000Boundary,statapath+"Interstate1979_2000Boundary.csv", "utf-8", None, "CSV")

#1981
#Add Dissolved Highway Files
IHS1981final_dissolved = iface.addVectorLayer(hwypath + "IHS1981final_dissolved.shp", "IHS1981final_dissolved", "ogr")
if not IHS1981final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1981_2000Boundary = processing.runalg('qgis:intersection', IHS1981final_dissolved, mastercountylayer, hwypath+"Interstate1981_2000Boundary.shp")

Interstate1981_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1981_2000Boundary.shp","Interstate1981_2000Boundary", "ogr")
if not Interstate1981_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1981_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1981_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1981_2000Boundary.updateFields()
idx = Interstate1981_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1981_2000Boundary.pendingFields() )

for f in Interstate1981_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1981_2000Boundary.updateFeature( f )

Interstate1981_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1981_2000Boundary,statapath+"Interstate1981_2000Boundary.csv", "utf-8", None, "CSV")

#1982
#Add Dissolved Highway Files
IHS1982final_dissolved = iface.addVectorLayer(hwypath + "IHS1982final_dissolved.shp", "IHS1982final_dissolved", "ogr")
if not IHS1982final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1982_2000Boundary = processing.runalg('qgis:intersection', IHS1982final_dissolved, mastercountylayer, hwypath+"Interstate1982_2000Boundary.shp")

Interstate1982_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1982_2000Boundary.shp","Interstate1982_2000Boundary", "ogr")
if not Interstate1982_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1982_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1982_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1982_2000Boundary.updateFields()
idx = Interstate1982_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1982_2000Boundary.pendingFields() )

for f in Interstate1982_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1982_2000Boundary.updateFeature( f )

Interstate1982_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1982_2000Boundary,statapath+"Interstate1982_2000Boundary.csv", "utf-8", None, "CSV")

#1986
#Add Dissolved Highway Files
IHS1986final_dissolved = iface.addVectorLayer(hwypath + "IHS1986final_dissolved.shp", "IHS1986final_dissolved", "ogr")
if not IHS1986final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1986_2000Boundary = processing.runalg('qgis:intersection', IHS1986final_dissolved, mastercountylayer, hwypath+"Interstate1986_2000Boundary.shp")

Interstate1986_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1986_2000Boundary.shp","Interstate1986_2000Boundary", "ogr")
if not Interstate1986_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1986_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1986_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1986_2000Boundary.updateFields()
idx = Interstate1986_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1986_2000Boundary.pendingFields() )

for f in Interstate1986_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1986_2000Boundary.updateFeature( f )

Interstate1986_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1986_2000Boundary,statapath+"Interstate1986_2000Boundary.csv", "utf-8", None, "CSV")

#1987
#Add Dissolved Highway Files
IHS1987final_dissolved = iface.addVectorLayer(hwypath + "IHS1987final_dissolved.shp", "IHS1987final_dissolved", "ogr")
if not IHS1987final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1987_2000Boundary = processing.runalg('qgis:intersection', IHS1987final_dissolved, mastercountylayer, hwypath+"Interstate1987_2000Boundary.shp")

Interstate1987_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1987_2000Boundary.shp","Interstate1987_2000Boundary", "ogr")
if not Interstate1987_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1987_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1987_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1987_2000Boundary.updateFields()
idx = Interstate1987_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1987_2000Boundary.pendingFields() )

for f in Interstate1987_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1987_2000Boundary.updateFeature( f )

Interstate1987_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1987_2000Boundary,statapath+"Interstate1987_2000Boundary.csv", "utf-8", None, "CSV")

#1989
#Add Dissolved Highway Files
IHS1989final_dissolved = iface.addVectorLayer(hwypath + "IHS1989final_dissolved.shp", "IHS1989final_dissolved", "ogr")
if not IHS1989final_dissolved:
    print "Layer failed to load!"

#Intersect Hwy Layer and County Boundary Layer
Interstate1989_2000Boundary = processing.runalg('qgis:intersection', IHS1989final_dissolved, mastercountylayer, hwypath+"Interstate1989_2000Boundary.shp")

Interstate1989_2000Boundary=iface.addVectorLayer(hwypath+"Interstate1989_2000Boundary.shp","Interstate1989_2000Boundary", "ogr")
if not Interstate1989_2000Boundary:
    print "Layer failed to load!"

#Calculate Distance of Each Segment
Interstate1989_2000Boundary.startEditing()

#step 1
seglength = QgsField( 'seglength', QVariant.Double )
Interstate1989_2000Boundary.dataProvider().addAttributes([seglength])
Interstate1989_2000Boundary.updateFields()
idx = Interstate1989_2000Boundary.fieldNameIndex( 'seglength' )

#step 2
e = QgsExpression( '$length' )
e.prepare( Interstate1989_2000Boundary.pendingFields() )

for f in Interstate1989_2000Boundary.getFeatures():
    f[idx] = e.evaluate( f )
    Interstate1989_2000Boundary.updateFeature( f )

Interstate1989_2000Boundary.commitChanges()

QgsVectorFileWriter.writeAsVectorFormat(Interstate1989_2000Boundary,statapath+"Interstate1989_2000Boundary.csv", "utf-8", None, "CSV")






